In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import random
import os
import torch
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

#### The Data

In [3]:
#df = pd.read_csv('../../data/anemia_synth_dataset_some_nans_unspecified_more_feats.csv')
#df = pd.read_csv('../../data/more_feats_0.2.csv')
#df= pd.read_csv('../../data/more_features/more_feats_new_labels_0.1.csv')
#df =pd.read_csv('../../data/more_features/more_feats_new_labels_0.1_noisy_0.6.csv')
df = pd.read_csv('../../data/more_features//more_feats_correlated_missing_3.csv')
#df = utils.balance_dataset(df, 8000)
df = df.fillna(-1)
df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,11.183192,187.573466,4.951674,-1.000000,316.593436,-1.000000,-1.000000,3.531295,1,-1.000000,-1.000000,96.721542,49.530542,23.186628,-1.0,-1.000000,57.247868,Hemolytic anemia
1,14.387445,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,125.249617,-1.000000,1,-1.000000,98.357508,112.758764,62.464566,7.168920,-1.0,-1.000000,-1.000000,No anemia
2,12.749357,-1.000000,-1.000000,6.179371,498.418768,76.759285,-1.000000,4.982859,1,1.022939,-1.000000,75.739552,72.072041,20.600875,-1.0,38.248071,32.068372,Iron deficiency anemia
3,11.508870,197.180945,1.200125,-1.000000,457.033309,102.900301,131.177927,3.355346,1,-1.000000,-1.000000,66.999185,18.353272,-1.000000,-1.0,34.526609,28.702050,Unspecified anemia
4,9.456656,427.952052,-1.000000,0.660252,-1.000000,104.543774,-1.000000,2.713693,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,28.369968,-1.000000,Vitamin B12/Folate deficiency anemia


In [4]:
df.isna().sum()

hemoglobin               0
ferritin                 0
ret_count                0
segmented_neutrophils    0
tibc                     0
mcv                      0
serum_iron               0
rbc                      0
gender                   0
creatinine               0
cholestrol               0
copper                   0
ethanol                  0
folate                   0
glucose                  0
hematocrit               0
tsat                     0
label                    0
dtype: int64

In [5]:
utils.get_dt_performance(df)

(0.7770714285714285,
 0.7458343221135431,
 0.8570010394453483,
 datetime.timedelta(microseconds=2001))

In [6]:
df.label.value_counts()

No anemia                               10000
Anemia of chronic disease                9756
Iron deficiency anemia                   9267
Unspecified anemia                       9033
Aplastic anemia                          9020
Vitamin B12/Folate deficiency anemia     9000
Hemolytic anemia                         8976
Inconclusive diagnosis                   4948
Name: label, dtype: int64

In [7]:
class_dict = constants.CLASS_DICT
df['label'] = df['label'].replace(class_dict)
X = df.iloc[:, 0:-1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56000, 17), (14000, 17), (56000,), (14000,))

In [8]:
action_list = list(class_dict.keys()) + [col  for col in df.columns if col!='label']
action_list

['No anemia',
 'Vitamin B12/Folate deficiency anemia',
 'Unspecified anemia',
 'Anemia of chronic disease',
 'Iron deficiency anemia',
 'Hemolytic anemia',
 'Aplastic anemia',
 'Inconclusive diagnosis',
 'hemoglobin',
 'ferritin',
 'ret_count',
 'segmented_neutrophils',
 'tibc',
 'mcv',
 'serum_iron',
 'rbc',
 'gender',
 'creatinine',
 'cholestrol',
 'copper',
 'ethanol',
 'folate',
 'glucose',
 'hematocrit',
 'tsat']

In [9]:
len(action_list)

25

In [10]:
df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,11.183192,187.573466,4.951674,-1.000000,316.593436,-1.000000,-1.000000,3.531295,1,-1.000000,-1.000000,96.721542,49.530542,23.186628,-1.0,-1.000000,57.247868,5
1,14.387445,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,125.249617,-1.000000,1,-1.000000,98.357508,112.758764,62.464566,7.168920,-1.0,-1.000000,-1.000000,0
2,12.749357,-1.000000,-1.000000,6.179371,498.418768,76.759285,-1.000000,4.982859,1,1.022939,-1.000000,75.739552,72.072041,20.600875,-1.0,38.248071,32.068372,4
3,11.508870,197.180945,1.200125,-1.000000,457.033309,102.900301,131.177927,3.355346,1,-1.000000,-1.000000,66.999185,18.353272,-1.000000,-1.0,34.526609,28.702050,2
4,9.456656,427.952052,-1.000000,0.660252,-1.000000,104.543774,-1.000000,2.713693,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,28.369968,-1.000000,1


#### Training 

In [11]:
# %%time
# timesteps = int(2e6)
# dqn_model = utils.stable_dqn3(X_train, y_train, timesteps, True, f'../../models/many_features/stable_dqn3_{timesteps}')
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

In [ ]:
#for steps in [int(6e6), int(6.5e6), int(7e6), int(7.5e6), int(8e6), int(8.5e6), int(9e6)]:
for steps in [int(11.5e6), int(12e6), int(12.5e6), int(13e6)]:
    #start_time = datetime.now()
    dqn_model = utils.stable_dqn3(X_train, y_train, steps, True, 
                                  f'../../models/many_features/0.1/with_correlated_fts/dqn3_by_type_missing_3_{steps}')
    #end_time = datetime.now()
    #print(f'The duration for {steps} steps is {end_time-start_time}')

using stable baselines 3
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.14     |
|    ep_rew_mean      | -0.8     |
|    exploration_rate | 0.765    |
|    success_rate     | 0.1      |
| time/               |          |
|    episodes         | 100000   |
|    fps              | 844      |
|    time_elapsed     | 336      |
|    total_timesteps  | 284488   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 10.1     |
|    n_updates        | 58621    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.32     |
|    ep_rew_mean      | -0.84    |
|    exploration_rate | 0.504    |
|    success_rate     | 0.11     |
| time/               |          |
|    episodes         | 200000   |
|    fps              | 754      |
|    t

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.96     |
|    ep_rew_mean      | -0.24    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.4      |
| time/               |          |
|    episodes         | 1600000  |
|    fps              | 549      |
|    time_elapsed     | 9798     |
|    total_timesteps  | 5379822  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0477   |
|    n_updates        | 1332455  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.58     |
|    ep_rew_mean      | -0.24    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.39     |
| time/               |          |
|    episodes         | 1700000  |
|    fps              | 548      |
|    time_elapsed     | 10474    |
|    total_timesteps  | 5747060  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.48     |
|    ep_rew_mean      | -0.3     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.39     |
| time/               |          |
|    episodes         | 3100000  |
|    fps              | 576      |
|    time_elapsed     | 18661    |
|    total_timesteps  | 10762585 |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.151    |
|    n_updates        | 2678146  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.39     |
|    ep_rew_mean      | -0.1     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.47     |
| time/               |          |
|    episodes         | 3200000  |
|    fps              | 578      |
|    time_elapsed     | 19229    |
|    total_timesteps  | 11122021 |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.09     |
|    ep_rew_mean      | -0.34    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.35     |
| time/               |          |
|    episodes         | 1300000  |
|    fps              | 525      |
|    time_elapsed     | 8067     |
|    total_timesteps  | 4237682  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.129    |
|    n_updates        | 1046920  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.79     |
|    ep_rew_mean      | -0.2     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.43     |
| time/               |          |
|    episodes         | 1400000  |
|    fps              | 449      |
|    time_elapsed     | 10199    |
|    total_timesteps  | 4579859  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.08     |
|    ep_rew_mean      | -0.32    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.35     |
| time/               |          |
|    episodes         | 2800000  |
|    fps              | 316      |
|    time_elapsed     | 29100    |
|    total_timesteps  | 9207820  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.193    |
|    n_updates        | 2289454  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.32     |
|    ep_rew_mean      | -0.4     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.3      |
| time/               |          |
|    episodes         | 2900000  |
|    fps              | 319      |
|    time_elapsed     | 29869    |
|    total_timesteps  | 9535453  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.92     |
|    ep_rew_mean      | -0.96    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.12     |
| time/               |          |
|    episodes         | 700000   |
|    fps              | 590      |
|    time_elapsed     | 3663     |
|    total_timesteps  | 2162614  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 3.83     |
|    n_updates        | 528153   |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.24     |
|    ep_rew_mean      | -0.98    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.09     |
| time/               |          |
|    episodes         | 800000   |
|    fps              | 541      |
|    time_elapsed     | 4594     |
|    total_timesteps  | 2489057  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.43     |
|    ep_rew_mean      | -0.32    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.39     |
| time/               |          |
|    episodes         | 2200000  |
|    fps              | 377      |
|    time_elapsed     | 19590    |
|    total_timesteps  | 7397707  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.104    |
|    n_updates        | 1836926  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4        |
|    ep_rew_mean      | -0.2     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.48     |
| time/               |          |
|    episodes         | 2300000  |
|    fps              | 381      |
|    time_elapsed     | 20302    |
|    total_timesteps  | 7754455  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.99     |
|    ep_rew_mean      | -0.88    |
|    exploration_rate | 0.562    |
|    success_rate     | 0.1      |
| time/               |          |
|    episodes         | 200000   |
|    fps              | 807      |
|    time_elapsed     | 741      |
|    total_timesteps  | 599175   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 8.73     |
|    n_updates        | 137293   |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.07     |
|    ep_rew_mean      | -0.96    |
|    exploration_rate | 0.318    |
|    success_rate     | 0.04     |
| time/               |          |
|    episodes         | 300000   |
|    fps              | 774      |
|    time_elapsed     | 1205     |
|    total_timesteps  | 933207   |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.46     |
|    ep_rew_mean      | -0.34    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.36     |
| time/               |          |
|    episodes         | 1700000  |
|    fps              | 549      |
|    time_elapsed     | 10035    |
|    total_timesteps  | 5511144  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.166    |
|    n_updates        | 1365285  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.43     |
|    ep_rew_mean      | -0.12    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.5      |
| time/               |          |
|    episodes         | 1800000  |
|    fps              | 537      |
|    time_elapsed     | 10918    |
|    total_timesteps  | 5868578  |
| train/              |          |
|    learning_rate  

#### Testing

In [ ]:
# training_env = utils.create_env(X_train, y_train)
# dqn_model = utils.load_dqn3('../../models/many_features/0.1/with_correlated_fts/dqn3_by_type_noisy_6_16000000', training_env)
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

In [ ]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

In [ ]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

In [ ]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

In [ ]:
test_df.y_pred.unique()

In [ ]:
test_df[test_df.y_pred==4]

#### Saving files

In [ ]:
# test_df.to_csv(f'../../test_dfs/many_features/0.1/correlated/test_df3_9000000.csv', index=False)
# success_df.to_csv(f'../../test_dfs/many_features/0.1/correlated/success_df3_9000000.csv', index=False)

#### Confusion matrix and classification report

In [ ]:
test_df = pd.read_csv('../../test_dfs/many_features/0.1/test_df3_6500000.csv')
test_df.head()

In [ ]:
utils.plot_classification_report(test_df['y_actual'], test_df['y_pred'])

In [ ]:
# def plot_confusion_matrix(y_actual, y_pred, save=False, filename=False):
#     from sklearn.metrics import confusion_matrix
#     cm = confusion_matrix(y_actual, y_pred)
#     cm_df = pd.DataFrame(cm, index = [0, 1, 2, 3, 4, 5, 6], columns = [0, 1, 2, 3, 4, 5, 6], dtype='object')
#     #cm_df = pd.DataFrame(cm, index = constants.CLASS_DICT.keys(), columns = constants.CLASS_DICT.keys())
#     plt.figure(figsize=(8, 6))
#     sns.heatmap(cm_df, annot=True)
#     plt.title('Confusion Matrix')
#     plt.ylabel('Actual Anemia')
#     plt.xlabel('Predicted Anemia')
#     plt.tight_layout()
#     if save:
#         plt.savefig(filename)
#     plt.show()
#     plt.close()

In [ ]:
utils.plot_confusion_matrix(test_df['y_actual'], test_df['y_pred'])